In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Alopecia/GSE18876'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Transcriptional Profile of Aging in Healthy Human Skin"
!Series_summary	"Gene expression changes were assessed from the non sun-exposed skin of the lower back of 98 healthy males aged 19-86. We show that contrary to previous thought, genome wide transcriptional activity does not display an exclusively linear correlation with ageing, but rather, in human skin, undergoes a period of significant transient change between 30 and 45 years of age. The identified transient transcriptional changes suggest a period of heightened metabolic activity and cellular damage mediated primarily through the actions of TP53 (tumour protein 53) and TNF (tumour necrosis factor). We also identified a subgroup of the population characterised by increased expression of a large group of hair follicle genes that correlates strongly with a younger age of onset and increasing severity of androgenetic alopecia."
!Series_overall_design	"Skin was collected from the lower back at 

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Determine if gene expression data is available
is_gene_available = True

# Analyze the sample characteristics for alopecia, age, and gender variables
sample_characteristics = {
    0: ['age: 19', 'age: 20', 'age: 21', 'age: 22', 'age: 23', 'age: 24', 'age: 25', 'age: 26', 'age: 27', 'age: 30', 'age: 31', 'age: 33', 'age: 34', 'age: 36', 'age: 38', 'age: 39', 'age: 41', 'age: 42', 'age: 43', 'age: 44', 'age: 45', 'age: 47', 'age: 49', 'age: 50', 'age: 51', 'age: 52', 'age: 53', 'age: 54', 'age: 55', 'age: 57'],
    1: ['tissue: skin']
}

# Check for age data
age_data_key_candidates = [key for key, values in sample_characteristics.items() if all('age:' in value for value in values)]
if age_data_key_candidates and len(set(value.split(':')[1].strip() for value in sample_characteristics[age_data_key_candidates[0]])) > 1:
    age_row = age_data_key_candidates[0]

# Check for alopecia data (trait)
# Assuming 'tissue: skin' does not indicate alopecia directly; we can't infer trait data
# Hence, setting trait_row to None as no alopecia data is available explicitly or implicitly

# Check for gender data
# As it's explicitly stated that this dataset involves only males, no distinct gender data is recorded
gender_row = None

# Define conversion functions
def convert_trait(value):
    return None  # No trait data available

def convert_age(value):
    try:
        return float(value.split(':')[1].strip())
    except:
        return None

def convert_gender(value):
    return None  # As per dataset, no gender data available

save_cohort_info('GSE18876', './preprocessed/Alopecia/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Alopecia', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Alopecia/trait_data/GSE18876.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


A new JSON file was created at: ./preprocessed/Alopecia/cohort_info.json
